In [ ]:
# -----********************-----

# Created Time: 2025/07/13

# Last updated: 2025/07/21

# Author: Yiyi He

### Use Case

# This notebook explores the application of Granger Causality and autoregressive models
# 1.

# -----********************-----

# Libraries

In [5]:
# Import libraries
import os
import warnings
from pathlib import Path
warnings.filterwarnings("ignore")

# Stats
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.api import ARDL
import statsmodels.api as sm
import numpy as np
from statsmodels.tsa.ardl import ardl_select_order
from statsmodels.tsa.stattools import grangercausalitytests

# Geo
from shapely.geometry import Point, Polygon
# import geopandas as gpd
import pandas as pd
pd.set_option('display.max_columns', 500)
pd.options.display.max_rows = 1000

# Plot
import matplotlib.pyplot as plt
# import seaborn as sns

# Processing
from tqdm import tqdm
import functools as ft

import warnings
warnings.filterwarnings("ignore")

# Granger Causality

## Test for stationarity

### ADF

In [36]:
# Hourly
home_dir = '/Users/yiyi/Library/CloudStorage/OneDrive-GeorgiaInstituteofTechnology(2)/Research/Energy_resilience/' # Office Mac
home_dir = '/Users/yiyihe/Library/CloudStorage/OneDrive-GeorgiaInstituteofTechnology/Research/Energy_resilience/' # Macbook Pro

hourly_df = pd.read_csv(home_dir + "01_data/processed/csv/hourly_519station_3weather.csv",
                        index_col=0)

# Generate a list of station ids
station_lst = list(set(hourly_df.station_id.unique()))

adf_res = {}
for s_id in tqdm(station_lst):
        # Extracting the dataframe for a/one station
        station_df = hourly_df[hourly_df['station_id'] == s_id].sort_values(by='datetime')
        # Extact time series
        pct_blackout_ts = station_df.pct_blackout.values # Target
        t2m_ts = station_df.t2m.values # Predictor
        tp_ts = station_df.tp.values # Predictor
        wind_speed_ts = station_df.wind_speed.values # Predictor

        adf_res[s_id] = [
                        adfuller(pct_blackout_ts)[1], # p-value
                        adfuller(t2m_ts)[1],          # p-value
                        adfuller(tp_ts)[1],           # p-value
                        adfuller(wind_speed_ts)[1]    # p-value
        ]
adf_res_df = pd.DataFrame.from_dict(adf_res, orient='index').reset_index()
adf_res_df.rename(columns={
    'index': 'station_id',
    0: 'pct_blackout_pvalue',
    1: 't2m_pvalue',
    2: 'tp_pvalue',
    3: 'wind_speed_pvalue'
    }, inplace=True)
adf_res_df.to_csv(home_dir + '01_data/processed/csv/ADF/hourly_ADF_test_pvalues.csv')

 94%|█████████▍| 488/519 [1:05:27<18:57, 36.71s/it]

## GC with hourly data

In [5]:
home_dir = '/Users/yiyi/Library/CloudStorage/OneDrive-GeorgiaInstituteofTechnology(2)/Research/Energy_resilience/'

hourly_df = pd.read_csv(home_dir + "01_data/processed/csv/hourly_519station_3weather.csv",
                        index_col=0)

Define function that runs the Granger Causality test on hourly station data. \
For each station, run the Granger Causality test with the following set up:\
 \
**Target variable**: \
'pct_blackout' \
**Predictor variables**: 
1. 't2m' (temperature) 
2. 'tp' (precipitation) 
3. 'wind_speed' (wind speed)

In [82]:
def run_gc_hourly(maxlag, target, predictor, station_lst, hourly_df):
    # Initiate an empty dictionary for storing test results
    gc_dic = {}

    # Iterate through all stations
    for s_id in tqdm(station_lst):
        # Extracting the dataframe for a/one station
        station_df = hourly_df[hourly_df['station_id'] == s_id].sort_values(by='datetime')
        try:
            # Check if the number of observations available at the station is sufficient for the gc model
            if len(station_df) <= maxlag + 2 + 1: # maxlag + num_variables (target, predictor) + 1
                print(f"Skipping Station {s_id} due to insufficient observations for maxlag={maxlag}")
                continue
            else:
                # Run granger causality test on station hourly data
                test_result = grangercausalitytests(
                    station_df[[target, predictor]], maxlag=maxlag, addconst=True, verbose=False)
                # Save test values
                F_test_p_values = [round(test_result[i+1][0]['ssr_ftest'][1],4) for i in range(maxlag)]
                Chi_squared_p_values = [round(test_result[i+1][0]['ssr_chi2test'][1],4) for i in range(maxlag)]
                p_values_min = np.min(F_test_p_values+Chi_squared_p_values)
                # Key: station id, Value: list of 1. minimum F/Chi p values 2. F-test p values for all lags 3. Chi-square test p-values for all lags
                gc_dic[s_id] = [p_values_min, F_test_p_values, Chi_squared_p_values]
        except ValueError:
            print(f"Skipping Station {s_id} due to insufficient observations for maxlag={maxlag}")
    # Convert to dataframe        
    gc_df = pd.DataFrame.from_dict(gc_dic, orient='index').reset_index()
    gc_df.rename(columns={
    'index':'station_id',
    0:f'{predictor}_p-value_min',
    1:f'{predictor}_f_p-value',
    2:f'{predictor}_Chi_p-value'
    }, inplace=True)
    return gc_df

In [83]:
# Set home directory
home_dir = '/Users/yiyi/Library/CloudStorage/OneDrive-GeorgiaInstituteofTechnology(2)/Research/Energy_resilience/'
# Read hourly data
hourly_df = pd.read_csv(home_dir + "01_data/processed/csv/hourly_519station_3weather.csv",
                        index_col=0)
# Set target variable
target = 'pct_blackout'
# Set max lag for all variables
maxlag = 120
# Generate a list of station ids
station_lst = list(set(hourly_df.station_id.unique()))
# List of predictor variables
predictor_lst = ['t2m', 'tp', 'wind_speed']

# Initiate empty dictionary to store test output dataframes
res_dic = {}
# Loop through the list of predictors
for predictor in predictor_lst:
    # Run gc test
    df_res = run_gc_hourly(maxlag, target, predictor, station_lst, hourly_df)
    # Save output dataframe in dictionary key: predictor, value: dataframe with test values
    res_dic[predictor] = df_res

# Join resulting dataframes for all predictors together
dfs = [res_dic[p] for p in predictor_lst]
df_joined= ft.reduce(lambda left, right: pd.merge(left, right, on='station_id'), dfs)
# Save output as new csv file
df_joined.to_csv(home_dir + f"01_data/processed/csv/granger_hourly_max{maxlag}_pvalue.csv")

 10%|█         | 54/519 [30:14<4:09:05, 32.14s/it]

Skipping Station 58 due to insufficient observations for maxlag=120


 18%|█▊        | 94/519 [48:09<2:24:04, 20.34s/it]

Skipping Station 103 due to insufficient observations for maxlag=120


 23%|██▎       | 119/519 [59:00<3:34:41, 32.20s/it]

...

 90%|████████▉ | 466/519 [3:39:14<08:09,  9.23s/it]  

Skipping Station 518 due to insufficient observations for maxlag=120


 98%|█████████▊| 509/519 [4:17:31<03:34, 21.41s/it]   

Skipping Station 563 due to insufficient observations for maxlag=120


100%|██████████| 519/519 [4:20:58<00:00, 30.17s/it]


## GC with daily data

In [ ]:
def run_gc_daily(maxlag, target, predictor, station_lst, daily_df):
    # Initiate an empty dictionary for storing test results
    gc_dic = {}

    # Iterate through all stations
    for s_id in tqdm(station_lst):
        # Extracting the dataframe for a/one station
        station_df = daily_df[daily_df['station_id'] == s_id].sort_values(by='date') # Sort by date
        try:
            # Check if the number of observations available at the station is sufficient for the gc model
            if len(station_df) <= maxlag + 2 + 1: # maxlag + num_variables (target, predictor) + 1
                print(f"Skipping Station {s_id} due to insufficient observations for maxlag={maxlag}")
                continue
            else:
                # Run granger causality test on station daily data
                test_result = grangercausalitytests(
                    station_df[[target, predictor]], maxlag=maxlag, addconst=True, verbose=False)
                # Save test values
                F_test_p_values = [round(test_result[i+1][0]['ssr_ftest'][1],4) for i in range(maxlag)]
                Chi_squared_p_values = [round(test_result[i+1][0]['ssr_chi2test'][1],4) for i in range(maxlag)]
                p_values_min = np.min(F_test_p_values+Chi_squared_p_values)
                # Key: station id, Value: list of 1. minimum F/Chi p values 2. F-test p values for all lags 3. Chi-square test p-values for all lags
                gc_dic[s_id] = [p_values_min, F_test_p_values, Chi_squared_p_values]
        except ValueError:
            print(f"Skipping Station {s_id} due to insufficient observations for maxlag={maxlag}")
    # Convert to dataframe        
    gc_df = pd.DataFrame.from_dict(gc_dic, orient='index').reset_index()
    gc_df.rename(columns={
    'index':'station_id',
    0:f'{predictor}_p-value_min',
    1:f'{predictor}_f_p-value',
    2:f'{predictor}_Chi_p-value'
    }, inplace=True)
    return gc_df

In [51]:
# Set home directory
home_dir = '/Users/yiyi/Library/CloudStorage/OneDrive-GeorgiaInstituteofTechnology(2)/Research/Energy_resilience/'
# Load station daily data
daily_df = pd.read_csv(home_dir + "01_data/processed/csv/daily_519station_13weather.csv", index_col=0)

# Target
target = "daily_blackout_minutes"
# Predictors
weather_variables = ['t2m', 'wind_speed', 'tp']
aggregations = ['median', 'mean', 'max']
predictor_lst = [w + "_" + a for w in weather_variables for a in aggregations]

# Create a list of unique station ids
station_id_lst = list(set(daily_df.station_id.unique()))

# Loop through multiple max lag values
for maxlag in [3, 5, 7, 15, 30, 45, 60]:
    # Initiate empty dictionary to store test output dataframes
    res_dic = {}
    # Loop through the list of predictors
    for predictor in predictor_lst:
        # Run gc test
        df_res = run_gc_daily(maxlag, target, predictor, station_lst, daily_df)
        # Save output dataframe in dictionary key: predictor, value: dataframe with test values
        res_dic[predictor] = df_res

    # Join resulting dataframes for all predictors together
    dfs = [res_dic[p] for p in predictor_lst]
    df_joined= ft.reduce(lambda left, right: pd.merge(left, right, on='station_id'), dfs)
    # Save output as new csv file
    df_joined.to_csv(home_dir + f"01_data/processed/csv/granger_daily_max{maxlag}_pvalue.csv")

 12%|█▏        | 64/519 [00:01<00:07, 61.21it/s]

Skipping Station 58 due to insufficient observations for maxlag=7


 20%|█▉        | 102/519 [00:01<00:06, 66.69it/s]

Skipping Station 103 due to insufficient observations for maxlag=7


 24%|██▍       | 127/519 [00:02<00:05, 67.97it/s]

...

Skipping Station 559 due to insufficient observations for maxlag=60
Skipping Station 560 due to insufficient observations for maxlag=60


 98%|█████████▊| 509/519 [09:27<00:10,  1.01s/it]

Skipping Station 563 due to insufficient observations for maxlag=60
Skipping Station 564 due to insufficient observations for maxlag=60


100%|██████████| 519/519 [09:38<00:00,  1.12s/it]


Skipping Station 572 due to insufficient observations for maxlag=60


# ARDL

Correlation between daily weather aggregate variables

In [54]:
# Set home directory
home_dir = '/Users/yiyi/Library/CloudStorage/OneDrive-GeorgiaInstituteofTechnology(2)/Research/Energy_resilience/'
# Load station daily data
daily_df = pd.read_csv(home_dir + "01_data/processed/csv/daily_519station_13weather.csv", index_col=0)

# Correlation
correlation_matrix = daily_df[['daily_blackout_minutes',
                                        't2m_min',
                                        't2m_median',
                                        't2m_max',
                                        'wind_speed_median',
                                        'wind_speed_max',
                                        'tp_median',
                                        'tp_max']].corr()
correlation_matrix.style.background_gradient(cmap='coolwarm')

,daily_blackout_minutes,t2m_min,t2m_median,t2m_max,wind_speed_median,wind_speed_max,tp_median,tp_max
daily_blackout_minutes,1.000000,0.020774,0.027144,0.024727,-0.002525,0.004347,0.034892,0.040650
t2m_min,0.020774,1.000000,0.911219,0.725396,0.265783,0.357374,0.192918,0.218318
t2m_median,0.027144,0.911219,1.000000,0.921124,0.187497,0.300769,0.007608,0.023848
t2m_max,0.024727,0.725396,0.921124,1.000000,0.102492,0.235172,-0.170036,-0.169363
wind_speed_median,-0.002525,0.265783,0.187497,0.102492,1.000000,0.872144,0.091777,0.104201
wind_speed_max,0.004347,0.357374,0.300769,0.235172,0.872144,1.000000,0.067437,0.087814
tp_median,0.034892,0.192918,0.007608,-0.170036,0.091777,0.067437,1.000000,0.812423
tp_max,0.040650,0.218318,0.023848,-0.169363,0.104201,0.087814,0.812423,1.000000


Given that the aggregate of one particular weather variable is highly correlated with one another. We decide to use the maximum for each weather variable: t2m_max, tp_max, and wind_speed_max

In [55]:
correlation_matrix = daily_df[['daily_blackout_minutes',
                                        't2m_max',
                                        'wind_speed_max',
                                        'tp_max']].corr()
correlation_matrix.style.background_gradient(cmap='coolwarm')

,daily_blackout_minutes,t2m_max,wind_speed_max,tp_max
daily_blackout_minutes,1.000000,0.024727,0.004347,0.040650
t2m_max,0.024727,1.000000,0.235172,-0.169363
wind_speed_max,0.004347,0.235172,1.000000,0.087814
tp_max,0.040650,-0.169363,0.087814,1.000000


The results show low correlation between the max of three weather variables.

## Daily

In [ ]:
def run_ARDL_findlag_daily(maxlag, target, predictor_lst, station_lst, ic, daily_df):
    # Initiate an empty dictionary
    res_dic = {}

    # Iterate through all stations
    for s_id in tqdm(station_lst):
            try:
                # Subset station data
                station_df = daily_df[daily_df['station_id'] == s_id].sort_values(by='date')
                # Ignore stations with less than 90 days/ 3 months worth of data
                if len(station_df) <= 90:
                     continue
                else:
                    # Find optimum lag and store station id with optimum lag in dictionary
                    sel_res = ardl_select_order(
                        station_df[target],
                        exog=station_df[predictors_lst],
                        maxlag=maxlag,
                        ic= ic,# string
                        maxorder=maxlag,
                        causal=True,
                        trend='ct'
                        )
                    # optimum lag values for endogenous variables
                    endo_res_lst = list(sel_res.bic.values[0][1].values())
                    # insert the optimum lag value for exdogenous variable
                    endo_res_lst.insert(0, sel_res.bic.values[0][0])
                    # store station id and optimum lag values in dictionary
                    res_dic[s_id] = endo_res_lst # key: station id, value:[pct_blackout_lag, t2m_lag, wind_speed_lag, tp_lag]
            except ValueError:
                 print('error')
    # Save results in dataframe             
    res_df = pd.DataFrame.from_dict(res_dic, orient='index')

    # Rename columns using meaningful names
    res_final_df = res_df.reset_index().rename(columns={**{'index': 'station_id', 0: 'blackout_minutes_lag'},
                                                        **{i+1: val+'_lag' for i, val in enumerate(predictors_lst)}})
    return res_final_df

In [ ]:
# Set home directory
home_dir = '/Users/yiyi/Library/CloudStorage/OneDrive-GeorgiaInstituteofTechnology(2)/Research/Energy_resilience/'
# Load station daily data
daily_df = pd.read_csv(home_dir + "01_data/processed/csv/daily_519station_13weather.csv", index_col=0)

# Loop through different maxlag values
for maxlag in [5, 10, 15, 30, 45, 60]: # unit: days
    print(f'I am working on maxlag={maxlag} now.')
    # Set target variable
    target = 'daily_blackout_minutes'
    # Set list of predictor variables
    predictors_lst = [
                    't2m_max',
                    'wind_speed_max',
                    'tp_max']
    # list of unique station ids
    station_lst = list(set(daily_df.station_id.unique()))
    # Information criterion
    ic = 'bic'
    # Run ARDL find lag
    result_df = run_ARDL_findlag_daily(maxlag, target, predictor_lst, station_lst, ic, daily_df)
    # Save output dataframe to csv
    result_df.to_csv(home_dir + f"01_data/processed/csv/ARDL_daily_optimum_{ic}_max{maxlag}.csv")

I am working on maxlag=5 now.


100%|██████████| 519/519 [07:03<00:00,  1.22it/s]


I am working on maxlag=10 now.


100%|██████████| 519/519 [3:31:12<00:00, 24.42s/it]  


I am working on maxlag=15 now.


...

 14%|█▍        | 75/519 [227:23:01<1495:02:50, 12122.00s/it]

error


 16%|█▋        | 85/519 [257:14:27<1252:46:20, 10391.66s/it]

error


 18%|█▊        | 94/519 [279:30:10<1039:52:54, 8808.41s/it] 